In [ ]:
import numpy as np

In [ ]:
# Process a path segment and return the steps for that segement, 
# e.g. U2 returns [(-1,0),(-2,0)]
def process_segment(pos, segment):
    direction = segment[0]
    value = int(segment[1:])
    
    lookup = {
        "U": (-1, 0),
        "D": (1, 0),
        "L": (0, -1),
        "R": (0, 1)
    }
    
    increment = lookup[direction]
    
    steps = []
    for i in range(0, value):
        pos = tuple(np.add(pos, increment))
        steps.append(pos)
    
    return steps

test = process_segment((0,0), "R10")
assert len(test) == 10
assert test[9] == (0,10)

# Process a single line and return the path
def process_line(value):
    segments = value.strip().split(",")
    start = (0,0)
    path = []
    for segment in segments:
        path += process_segment(start, segment)
        start = path[-1]
    
    return path

path = process_line("L10,U10,R10,D10")
assert len(path) == 40
assert path[9] == (0, -10)
assert path[-1] == (0, 0)

# Find intersects between two lines
def find_intersects(line1, line2):
    set1 = set(line1)
    set2 = set(line2)
    return set1 & set2

intersects = find_intersects(process_line("R10"), process_line("U2,R5,D5"))
assert len(intersects) == 1
assert (0,5) in intersects

intersects = find_intersects(process_line("R10,D2,L10"), process_line("U2,R5,D5"))
assert len(intersects) == 2
assert (0,5) in intersects
assert (2,5) in intersects


# Calculate differences between point and other coordinates
def manhattan_distance(p1, p2):
    dist = np.subtract(p1, p2)
    dist = np.abs(dist)
    dist = sum(dist)
    return dist

assert manhattan_distance((0,0),(5,5)) == 10
assert manhattan_distance((0,0),(5,-5)) == 10

# Find the closes point to origin
def find_closest(origin, points):
    shortest = None
    point = None
    for p in points:
        dist = manhattan_distance(origin, p)
        if shortest == None or dist < shortest:
            shortest = dist
            point = p
            
    return point, shortest

point, distance = find_closest((0,0), [(10,10), (2,2), (5,5)])
assert distance == 4
assert point == (2,2)

def find_closest_intersect(input1, input2):
    line1 = process_line(input1)
    line2 = process_line(input2)
    intersects = find_intersects(line1, line2)
    return find_closest((0,0), intersects)
    

In [ ]:
point, distance = find_closest_intersect("R8,U5,L5,D3", "U7,R6,D4,L4")
assert point == (-3,3)
assert distance == 6


point, distance = find_closest_intersect("R75,D30,R83,U83,L12,D49,R71,U7,L72",
                                         "U62,R66,U55,R34,D71,R55,D58,R83")

assert point == (-4, 155)
assert distance == 159

point, distance = find_closest_intersect("R98,U47,R26,D63,R33,U87,L62,D20,R33,U53",
                                         "U98,R91,D20,R16,D67,R40,U7,R15,U6,R7")

assert distance == 135

In [ ]:
with open("03-input.txt", "rt") as FILE:
    values = FILE.readlines()
    
find_closest_intersect(values[0], values[1])

# Part 2

In [ ]:
line1 = process_line("R8,U5,L5,D3")
line2 = process_line("U7,R6,D4,L4")
intersects = find_intersects(line1, line2)
intersects

# Calculates how many steps on the line to each of the given points
def journey_to_points(line, points):
    steps = {}
    for p in points:
        steps[p] = line.index(p) + 1
    return steps

steps1 = journey_to_points(line1, intersects)
assert steps1[(-5,6)] == 15
assert steps1[(-3,3)] == 20

steps2 = journey_to_points(line2, intersects)
assert steps2[(-5,6)] == 15
assert steps2[(-3,3)] == 20

# Calculate combined distances
def find_combined_steps_to_intersection(steps1, steps2):
    steps = {}
    for p in steps1.keys():
        dist = steps1[p] + steps2[p]
        steps[p] = dist
    return steps    

steps = find_combined_steps_to_intersection(steps1, steps2)
assert steps[(-5,6)] == 30
assert steps[(-3,3)] == 40

# Find the point with the smallest distance
# Based on a dict of {pos: distance, ...}
def find_smallest_step(steps):
    key_min = min(steps.keys(), key=(lambda k: steps[k]))
    return key_min, steps[key_min]

point, distance = find_smallest_step(steps)
assert point == (-5, 6)
assert distance == 30

def find_smallest_step_for_journeys(input1, input2):
    line1 = process_line(input1)
    line2 = process_line(input2)
    intersects = find_intersects(line1, line2)
    steps1 = journey_to_points(line1, intersects)
    steps2 = journey_to_points(line2, intersects)
    steps = find_combined_steps_to_intersection(steps1, steps2)
    return find_smallest_step(steps)


In [ ]:
point, distance = find_smallest_step_for_journeys(
    "R75,D30,R83,U83,L12,D49,R71,U7,L72",
    "U62,R66,U55,R34,D71,R55,D58,R83")

assert distance == 610

point, distance = find_smallest_step_for_journeys(
    "R98,U47,R26,D63,R33,U87,L62,D20,R33,U53",
    "U98,R91,D20,R16,D67,R40,U7,R15,U6,R7")

assert distance == 410
